# Topic modelling of news headlines for prediction of news category with 7 topics

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import wordcloud

import pyLDAvis.lda_model

## Load train set

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Extract features from 'title' 

In [7]:
train[train['title_cleaned'].isnull()]

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
39381,2023-02-16 15:30:15,Champions League,Liveticker und Highlightvideos,Missing,NaN,ZDF heute,Champions League - Liveticker und Highlightvid...,NaN,Champion Liveticker Highlightvideos
66356,2023-09-21 00:30:05,Champions League,Die wichtigsten Spiele im Video.,Missing,NaN,ZDF heute,Champions League - Die wichtigsten Spiele im V...,NaN,wichtig Spiel Video


In [8]:
# TODO workaround for missing title_cleaned -> TODO Solve this in NLP preprocessing!
train['title_cleaned'] = train['title_cleaned'].fillna(train['title'].apply(lambda x: x.lower()))

In [9]:
train[train['title_cleaned'].isnull()]

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [12]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [13]:
dtm

<67513x21412 sparse matrix of type '<class 'numpy.int64'>'
	with 330205 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.decomposition import LatentDirichletAllocation

In [15]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [16]:
len(cv.get_feature_names_out())

21412

In [17]:
cv.get_feature_names_out()

array(['000', '034', '039', ..., 'войны', 'российские', 'украинцы'],
      dtype=object)

In [18]:
len(lda.components_)

7

In [19]:
lda.components_

array([[ 0.14342263, 19.20866123,  2.10579852, ...,  0.14285737,
         0.14285737,  0.14285766],
       [ 1.51820379,  7.75579704,  0.20856291, ...,  2.14285544,
         2.14285544,  0.14285779],
       [12.73526902, 10.37410041, 23.28693696, ...,  0.14285742,
         0.14285742,  0.14285775],
       ...,
       [ 0.14291439, 14.18302458, 21.06086724, ...,  0.14285744,
         0.14285744,  0.14285782],
       [ 0.14326918,  4.85435504,  0.14298397, ...,  0.14285742,
         0.14285742,  0.14285777],
       [ 2.17404299, 11.2785756 ,  0.14292771, ...,  0.14285745,
         0.14285745,  2.14285337]])

In [20]:
len(lda.components_[0])

21412

### Show most important words of first extracted topic

In [21]:
first_topic = lda.components_[0]

In [22]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([20969, 15342, 16516, ..., 15091, 10150, 18325], dtype=int64)

In [23]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [24]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

liveblog
selenskyj
angriff
kiew
lage
putin
russisch
russland
krieg
ukraine


### Show most important words of first extracted topic

In [25]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['stimme', 'invasion', 'ukrainisch', 'moskau', 'nato', 'liveblog', 'selenskyj', 'angriff', 'kiew', 'lage', 'putin', 'russisch', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['johnson', 'großbritannien', 'eu', 'fdp', 'spd', 'panzer', 'land', 'faeser', 'streit', 'habeck', 'wahrheit', 'lindner', 'regierung', 'ampel', 'euro']


Die TOP-15 Wörter für das Thema #2
['usa', 'biden', 'million', 'fußball', 'us', 'gut', 'welt', 'taiwan', 'groß', 'analyse', 'berlin', 'wm', 'deutschland', 'scholz', 'china']


Die TOP-15 Wörter für das Thema #3
['podcast', 'nordkorea', 'brauchen', 'russland', 'haft', 'putin', 'bahn', 'ex', 'partei', 'türkei', 'klima', 'wahl', 'israel', 'afd', 'deutsch']


Die TOP-15 Wörter für das Thema #4
['retten', 'mindestens', 'merz', 'europa', 'steingart', 'gabor', 'spd', 'sanktion', 'gas', 'gastbeitrag', 'frau', 'grüne', 'cdu', 'tote', 'eu']


Die TOP-15 Wörter für das Thema #5
['deutsch', 'präsident', 'mensch', 'news'

### Get topics with highest probability for news items in train data

In [26]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [27]:
topic_results[0].round(5)

array([0.01796, 0.34617, 0.01789, 0.25018, 0.18461, 0.16531, 0.01787])

In [28]:
topic_results[0].argmax()

1

In [29]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [30]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...,1
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...,1
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,1


## Visualize model

In [31]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      13.659904 -17.311047       1        1  17.310420
2     -19.863859 -16.828156       2        1  14.593923
5      -3.588743 -46.077969       3        1  14.189605
1      -2.991294  -7.395037       4        1  14.116161
4     -20.184076 -36.156700       5        1  13.414833
3      13.293283 -36.642715       6        1  13.191746
6      -3.280650 -26.731941       7        1  13.183313, topic_info=            Term         Freq        Total Category  logprob  loglift
18325    ukraine  4529.000000  4529.000000  Default  30.0000  30.0000
10150      krieg  2624.000000  2624.000000  Default  29.0000  29.0000
15087   russisch  1378.000000  1378.000000  Default  28.0000  28.0000
3583      corona  1139.000000  1139.000000  Default  27.0000  27.0000
15091   russland  1759.000000  1759.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
14565  regierung   130.992178   514.006381   Topic7  -5.8128   0.6591
8879      israel   120.704755   421.296239   Topic7  -5.8946   0.7762
6604       geben   127.138272   568.325794   Topic7  -5.8427   0.5288
8226        hoch   111.315791   444.598859   Topic7  -5.9756   0.6414
1778    baerbock   111.325472   479.169871   Topic7  -5.9755   0.5666

[445 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
8          6  0.982482         11km
63         5  0.971093           49
138        2  0.981451          abe
418        2  0.161821          afd
418        4  0.106733          afd
...      ...       ...          ...
20777      2  0.991615       zahlen
20815      7  0.993314  zehntausend
21104      2  0.978737       zverev
21259      4  0.993008         öpnv
21275      1  0.995049    überblick

[872 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 6, 2, 5, 4, 7])

## Save train set with topics and dump model

In [32]:
#Save result to csv
train.to_csv('train_lda_topics_7.csv')

In [33]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv.jl')

['models/cv.jl']

In [34]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_7_topics.jl')

['models/lda_model_7_topics.jl']